In [1]:
import pandas as pd 
import glob
import cv2 as cv
import random
import os

import matplotlib.pyplot as plt
import numpy as np
import random
from PIL import Image
import PIL.ImageOps    

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import torchvision.utils
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from sklearn.metrics import f1_score, accuracy_score


from torch.cuda.amp import autocast
from torch.cuda.amp import GradScaler
from tqdm.auto import tqdm
import timm
import math
from sklearn.model_selection import train_test_split
import pytorch_model_summary



c:\Users\EHmin\anaconda3\envs\EHmin\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Make DataFrame

## columns: review_img_path, product_img_path, label
---

In [2]:
all_product_img_list = glob.glob('./masked_data/product_img/*')
all_product_img_list = sorted(all_product_img_list)
print(all_product_img_list)

['./masked_data/product_img\\0.jpg', './masked_data/product_img\\1.jpg', './masked_data/product_img\\10.jpg', './masked_data/product_img\\100.jpg', './masked_data/product_img\\12.jpg', './masked_data/product_img\\13.jpg', './masked_data/product_img\\14.jpg', './masked_data/product_img\\15.jpg', './masked_data/product_img\\20.jpg', './masked_data/product_img\\25.jpg', './masked_data/product_img\\28.jpg', './masked_data/product_img\\30.jpg', './masked_data/product_img\\33.jpg', './masked_data/product_img\\34.jpg', './masked_data/product_img\\35.jpg', './masked_data/product_img\\36.jpg', './masked_data/product_img\\37.jpg', './masked_data/product_img\\39.jpg', './masked_data/product_img\\4.jpg', './masked_data/product_img\\40.jpg', './masked_data/product_img\\41.jpg', './masked_data/product_img\\42.jpg', './masked_data/product_img\\43.jpg', './masked_data/product_img\\45.jpg', './masked_data/product_img\\46.jpg', './masked_data/product_img\\48.jpg', './masked_data/product_img\\49.jpg', '.

In [3]:
review_img_path = []
product_img_path = []
label = []
for product_img in all_product_img_list:
    glob_img_list = []
    str = product_img.split("\\")[1] # for window
    # str = product_img.split("/")[3] # for linux
    
    str = str.split(".")[0]
    glob_img_list = glob.glob('./masked_data/review_img/'+str +'_review_img/O/*.jpg')
    review_img_path = review_img_path + glob_img_list
    product_img_path = product_img_path + [product_img  for i in range(len(glob_img_list))]

In [4]:
print(len(product_img_path))
print(len(review_img_path))


18749
18749


In [5]:
for i in range(len(review_img_path)):
    random_number = random.randint(0, 1)
    if random_number == 0:
        label.append(0)
    else:
        review_num = review_img_path[i].split('\\')[1] # for Window 
        # review_num = review_img_path[i].split('/')[3] # for Linux
        
        review_num = review_num.split("_")[0]
        random_number = random.randint(0, len(all_product_img_list) - 1)
        while review_num == random_number:
            random_number = random.randint(0, len(all_product_img_list) - 1)
        product_img_path[i] = all_product_img_list[random_number]
        label.append(1)

In [6]:
df = pd.DataFrame(columns=['review_img_path','product_img_path', 'label'])
df['review_img_path'] = review_img_path
df['product_img_path'] = product_img_path
df['label'] = label

df

,review_img_path,product_img_path,label
0,./masked_data/review_img/0_review_img/O\1.jpg,./masked_data/product_img\0.jpg,0
1,./masked_data/review_img/0_review_img/O\107.jpg,./masked_data/product_img\20.jpg,1
2,./masked_data/review_img/0_review_img/O\109.jpg,./masked_data/product_img\0.jpg,0
3,./masked_data/review_img/0_review_img/O\111.jpg,./masked_data/product_img\0.jpg,0
4,./masked_data/review_img/0_review_img/O\119.jpg,./masked_data/product_img\0.jpg,0
...,...,...,...
18744,./masked_data/review_img/98_review_img/O\93.jpg,./masked_data/product_img\68.jpg,1
18745,./masked_data/review_img/98_review_img/O\95.jpg,./masked_data/product_img\98.jpg,0
18746,./masked_data/review_img/98_review_img/O\96.jpg,./masked_data/product_img\98.jpg,0
18747,./masked_data/review_img/98_review_img/O\97.jpg,./masked_data/product_img\37.jpg,1


---

# Siames Net

In [7]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [8]:
CFG = {
    'IMG_SIZE':256,
    'EPOCHS':15,
    'LEARNING_RATE':3e-3,
    # 'LEARNING_RATE':10,
    'BATCH_SIZE':8,
    'SEED':41
}

In [9]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = 'a'
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [10]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=CFG['SEED'])


In [11]:
class SiameseNetworkDataset(Dataset):
    def __init__(self,review_img_path,product_img_path,label,transform=None):
        self.review_img_path = review_img_path
        self.product_img_path = product_img_path
        self.label = label
        self.transform = transform
        
    def __getitem__(self,index):
        review_img = cv.imread(self.review_img_path[index])
        product_img = cv.imread(self.product_img_path[index])
        review_img = cv.resize(review_img, (CFG['IMG_SIZE'], CFG['IMG_SIZE']))
        product_img = cv.resize(product_img, (CFG['IMG_SIZE'], CFG['IMG_SIZE']))
        image = cv.hconcat([review_img, product_img])

        if self.transform is not None:
            image  = self.transform(image=image)['image']
    
        return image, self.label[index]
    
    def __len__(self):
        return len(self.review_img_path)

In [12]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            ToTensorV2()
                            ])


In [13]:
train_dataset = SiameseNetworkDataset(train["review_img_path"].values, train["product_img_path"].values, train["label"].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

val_dataset = SiameseNetworkDataset(val["review_img_path"].values, val["product_img_path"].values, val["label"].values, train_transform)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [14]:
avail_pretrained_models = timm.list_models(pretrained=True)
print(avail_pretrained_models)

['adv_inception_v3', 'bat_resnext26ts', 'beit_base_patch16_224', 'beit_base_patch16_224_in22k', 'beit_base_patch16_384', 'beit_large_patch16_224', 'beit_large_patch16_224_in22k', 'beit_large_patch16_384', 'beit_large_patch16_512', 'beitv2_base_patch16_224', 'beitv2_base_patch16_224_in22k', 'beitv2_large_patch16_224', 'beitv2_large_patch16_224_in22k', 'botnet26t_256', 'cait_m36_384', 'cait_m48_448', 'cait_s24_224', 'cait_s24_384', 'cait_s36_384', 'cait_xs24_384', 'cait_xxs24_224', 'cait_xxs24_384', 'cait_xxs36_224', 'cait_xxs36_384', 'coat_lite_mini', 'coat_lite_small', 'coat_lite_tiny', 'coat_mini', 'coat_tiny', 'coatnet_0_rw_224', 'coatnet_1_rw_224', 'coatnet_bn_0_rw_224', 'coatnet_nano_rw_224', 'coatnet_rmlp_1_rw_224', 'coatnet_rmlp_2_rw_224', 'coatnet_rmlp_nano_rw_224', 'coatnext_nano_rw_224', 'convit_base', 'convit_small', 'convit_tiny', 'convmixer_768_32', 'convmixer_1024_20_ks9_p14', 'convmixer_1536_20', 'convnext_atto', 'convnext_atto_ols', 'convnext_base', 'convnext_base_384_in

In [15]:
class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()
        # self.backbone = timm.create_model('maxvit_tiny_tf_512', pretrained=True)
        self.backbone = timm.create_model('efficientnet_b0', pretrained=False)
        self.classifier1 = nn.Linear(1000, 10)
        self.classifier2 = nn.Linear(10, 2)
        
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier1(x)
        x = self.classifier2(x)
        return F.log_softmax(x, dim=1)

In [16]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []
    correct = 0
    total = 0

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.type(torch.LongTensor).to(device)
            
            pred = model(imgs)
            # print(pred.shape)
            
            loss = criterion(pred, labels)
            
            preds += pred.detach().argmax(1).cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
            
            # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html
            
        
        _val_loss = np.mean(val_loss)
        _val_score = accuracy_score(true_labels, preds, normalize= True)
        
    
    return _val_loss, _val_score

In [17]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model = model.to(device)
    # criterion = nn.BCEWithLogitsLoss.to(device)
    # criterion = nn.BCEWithLogitsLoss(weight=None, size_average=None, reduce=None, reduction='mean', pos_weight=None).to(device)
    # criterion = nn.CrossEntropyLoss().to(device)
    criterion = nn.NLLLoss(reduction="sum").to(device)
    scaler = GradScaler()
    
    best_score = 0
    best_model = None
    
    for epoch in range(0, CFG['EPOCHS']):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.type(torch.LongTensor).to(device)
            
            optimizer.zero_grad()
            output = model(imgs)
            loss = criterion(output, labels)
            # with autocast():
            #     output = model(imgs)
            #     print(output)
            #     loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            # scaler.scale(loss).backward()
            # scaler.step(optimizer)
            # scaler.update()
            # print(loss.item())
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val accuracy score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
    
    return best_model

In [18]:
model = BaseModel()
model.load_state_dict(torch.load('./eff0b_10.pt'))
model.eval()
print(pytorch_model_summary.summary(model, torch.zeros(8,3,256,256),max_depth=None, show_parent_layers=True, show_input=True))

----------------------------------------------------------------------------------------------------------------------------------------------
                                                  Parent Layers             Layer (type)           Input Shape         Param #     Tr. Param #
                                         BaseModel/EfficientNet                 Conv2d-1      [8, 3, 256, 256]             864             864
                          BaseModel/EfficientNet/BatchNormAct2d               Identity-2     [8, 32, 128, 128]               0               0
                          BaseModel/EfficientNet/BatchNormAct2d                   SiLU-3     [8, 32, 128, 128]               0               0
                  BaseModel/EfficientNet/DepthwiseSeparableConv                 Conv2d-4     [8, 32, 128, 128]             288             288
   BaseModel/EfficientNet/DepthwiseSeparableConv/BatchNormAct2d               Identity-5     [8, 32, 128, 128]               0               0

In [19]:
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=3e-5, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

100%|██████████| 704/704 [00:38<00:00, 18.36it/s]


Epoch [0], Train Loss : [4.64291] Val Loss : [4.50620] Val accuracy score : [0.70187]


100%|██████████| 704/704 [00:39<00:00, 18.01it/s]


Epoch [1], Train Loss : [4.58621] Val Loss : [4.44923] Val accuracy score : [0.70240]


100%|██████████| 704/704 [00:37<00:00, 18.58it/s]


...

Epoch [12], Train Loss : [3.51162] Val Loss : [4.77844] Val accuracy score : [0.70133]


100%|██████████| 704/704 [00:50<00:00, 14.02it/s]


Epoch [13], Train Loss : [3.33263] Val Loss : [4.98154] Val accuracy score : [0.69458]
Epoch 00014: reducing learning rate of group 0 to 7.5000e-04.


100%|██████████| 704/704 [00:50<00:00, 14.05it/s]

Epoch [14], Train Loss : [3.08029] Val Loss : [5.30304] Val accuracy score : [0.69707]


In [20]:
torch.save(model.state_dict(), "./eff0b_25.pt")